# EXPORTACION DE TABLAS A CSV

In [ ]:
from sqlalchemy import create_engine, inspect

In [ ]:
# Database conexion
host = "127.0.0.1"                 
user = "admin"                      
password = "admin"                  
database = "ctk_201801_202007"      
port = 3306                        

# Start sqlalchemy engine
path_conexion = f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}'
conexion = create_engine(path_conexion)

# Listado de todas las tablas de la base de datos
inspector = inspect(conexion)
lista_tablas = inspector.get_table_names()

In [ ]:
len(lista_tablas), lista_tablas[:10], lista_tablas[-10:]

### Convert Transaction Tables

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
a = np.arange(2018, 2021, 1, dtype=int)
b = np.arange(1, 13, 1, dtype=int)
sis_tables = [f'sis_transa_{y}_{m}' for y in a for m in b]

In [ ]:
sis_tables[:5]

In [ ]:
%%time
for t in sis_tables:
    print(f'Cargando y creando tabla {t}')
    df = pd.read_sql_table(t, conexion)
    df.to_csv(f'./data_csv/{t}.csv', index=False)

In [ ]:
df_transactions = pd.concat([pd.read_csv(f'./data_csv/{t}.csv') for t in sis_tables[:-5]], ignore_index=True)

In [ ]:
df_transactions.to_csv(f'./data_csv/sis_transa_201801_202007.csv', index=False)

### Convert other related tables

In [ ]:
fs_tables = [lista_tablas[i] for i in range(0, len(lista_tablas)) if lista_tablas[i].startswith('fs_')]

In [ ]:
%%time
for t in fs_tables:
    print(f'Cargando y creando tabla {t}')
    df = pd.read_sql_table(t, conexion)
    df.to_csv(f'{t}.csv', index=False)

### ZIP CSV FILES

In [ ]:
# Comprimidp en ZIP de archivos csv
import os
import zipfile
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))
            
zipf = zipfile.ZipFile('./data_csv.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('./data_csv/', zipf)
zipf.close()  # Importante hacerlo